In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from vexa import VexaAPI
vexa = VexaAPI()
await vexa.get_user_info()

Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


{'id': 'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210',
 'email': '2280905@gmail.com',
 'is_allowed_send_init_message': True,
 'username': 'string',
 'first_name': 'string',
 'last_name': 'string',
 'image': None,
 'created_timestamp': '2024-05-28T11:16:50.894711Z',
 'updated_timestamp': '2024-11-05T19:10:10.736578Z'}

In [3]:
from vexa import VexaAPI
from pydantic_models import MeetingExtraction, EntityExtraction, MeetingNameAndSummary
from psql_helpers import get_meetings_by_user_id

In [4]:
meetings, total = await get_meetings_by_user_id(vexa.user_id)

# Get a specific meeting (e.g., the latest one)
meeting = meetings[50]  # Or select specific meeting by index
meeting_id = meeting['meeting_id']

# Get transcript for the meeting
transcription = await vexa.get_transcription(meeting_session_id=meeting_id)
df, formatted_input, start_datetime, speakers, transcript = transcription


In [5]:
formatted_input

"Meeting Metadata:\nStart Date and Time: 2024-10-03 17:02:52\nSpeakers (Always  reference the names EXACTLY as provided with NO changes letter by letter):\n  DG: Dmitriy Grankin\n  ML: Matt Lewis\n\nMeeting Transcript:\n\nML (00:00):  аккаунт. Что говоришь? Я старался через какой-то чужой аккаунт войти. Может быть Не видел, что я там ждал.\nML (00:00):  Бывает такое.\nDG (00:03):  Извини, я только\nML (00:08):  обратно с пробежки. Я возьму полотенчик, ладно? Давай, может, хочешь\nDG (00:10):  сходи в душ, попозже созвонимся.\nDG (00:12):  Да пять минут\nML (00:12):  Есть, есть, есть.\nML (00:14):  Конечно, давай, давай.\nDG (00:15):  Давай, давай.\nDG (00:15):  Спасибо.\nDG (00:16):  Окей, пока.\nDG (00:17):  Пока.\nDG (10:24):  нет я раньше каждый день бегал я сейчас наверно три-четыре раза в неделю где-то также у меня тут такие стали часто что я фитнес-зал Я не хожу на фитнес.\nML (10:47):  занимаешься на улице да да я люблю в природе так некрасиво но я своей генетики очень худенький

In [6]:
import asyncio

In [7]:

# Extract discussion points and entities
extraction_tasks = [
    MeetingExtraction.extract(formatted_input),
    EntityExtraction.extract(formatted_input)
]
discussion_points_df, topics_df = await asyncio.gather(*extraction_tasks)

In [8]:
import pandas as pd

In [9]:

# Process and combine the dataframes
discussion_points_df['model'] = 'MeetingExtraction'
topics_df['model'] = 'EntityExtraction'

# Rename columns to match schema
discussion_points_df = discussion_points_df.rename(columns={'item': 'topic_name', 'type': 'topic_type'})
topics_df = topics_df.rename(columns={'entity': 'topic_name', 'type': 'topic_type'})

# Combine dataframes
points_df = pd.concat([discussion_points_df, topics_df]).reset_index(drop=True)
final_df = points_df.reset_index().rename(columns={'index': 'summary_index'})

In [15]:
pd.options.display.max_colwidth = 1000

In [16]:
final_df

,summary_index,topic_name,topic_type,summary,details,speaker,model
0,0,Проблемы с доступом к аккаунту,проблема,"Матт Льюис не смог войти в аккаунт, использовав чужой аккаунт.","Матт Льюис пытался войти в аккаунт через чужой логин, что привело к проблемам с доступом.",Матт Льюис,MeetingExtraction
1,1,Физическая активность и фитнес,обсуждение,Дмитрий Гранкин и Матт Льюис обсудили свои физические активности и изменения в весе.,"Дмитрий Гранкин бегает 3-4 раза в неделю, а Матт Льюис заметил увеличение веса на 6 кг, что повлияло на его бег.",Дмитрий Гранкин,MeetingExtraction
2,2,Маркетинг и привлечение клиентов в Бразилии,решение,Дмитрий Гранкин начал маркетинг в Бразилии для привлечения клиентов.,"Дмитрий Гранкин провел исследование конкурентов и заметил, что многие работают только в США, что создает возможность для выхода на бразильский рынок.",Дмитрий Гранкин,MeetingExtraction
3,3,SEO и создание контента,идея,Дмитрий Гранкин разработал движок для генерации статей на основе SEO.,"Дмитрий Гранкин собирает ключевые слова и создает статьи, которые соответствуют запросам пользователей, что может привести к увеличению трафика.",Дмитрий Гранкин,MeetingExtraction
4,4,Открытие счета в Interactive Brokers,проблема,Дмитрий Гранкин не смог открыть счет в Interactive Brokers.,"Дмитрий Гранкин столкнулся с трудностями при открытии счета, так как компания требовала повторные подтверждения.",Дмитрий Гранкин,MeetingExtraction
5,5,Создание блога для привлечения клиентов,задача,Матт Льюис работает над улучшением своего блога для привлечения клиентов.,"Матт Льюис создает контент и видео, чтобы увеличить трафик на своем сайте и привлечь клиентов.",Матт Льюис,MeetingExtraction
6,6,Проблемы с мобильной связью и документами,проблема,Матт Льюис обсуждает необходимость utility bill для открытия счета.,"Матт Льюис ищет способы предоставить необходимые документы для открытия счета в банке, включая возможность использования мобильной связи.",Матт Льюис,MeetingExtraction
7,7,Дмитрий Гранкин,person,Дмитрий Гранкин обсуждает свои занятия спортом и маркетингом.,"Дмитрий делится своим опытом в фитнесе и маркетинге, включая инфлюенс маркетинг в Бразилии.",Дмитрий Гранкин,EntityExtraction
8,8,Мэтт Льюис,person,Мэтт Льюис делится своим опытом работы с клиентами и стратегиями маркетинга.,Мэтт обсуждает свои подходы к привлечению клиентов и использованию технологий для улучшения бизнеса.,Мэтт Льюис,EntityExtraction
9,9,фитнес-зал,concept,"Фитнес-зал упоминается как место, где Дмитрий Гранкин не занимается.","Дмитрий говорит о том, что он не ходит в фитнес-зал, а предпочитает заниматься на улице.",Дмитрий Гранкин,EntityExtraction


In [17]:
from qdrant_search import QdrantSearchEngine
self = QdrantSearchEngine()

/home/dima/anaconda3/envs/langchain/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [19]:
general_context_df = await self.sample_user_discussions(user_id=vexa.user_id, sample_size=100)



In [21]:
general_context_df = general_context_df[['topic_name', 'summary','details','timestamp','speaker_name','timestamp']]

In [24]:
general_context_df = general_context_df[~general_context_df['speaker_name'].str.contains('nan')]

In [30]:

# Generate meeting name and summary
meeting_name_and_summary = await MeetingNameAndSummary.extract(
    formatted_input, 
    final_df.drop(columns=['model']).to_markdown(index=False),
    general_context_df.to_markdown(index=False)
    
)

In [32]:
meeting_name_and_summary.meeting_name

'Проблемы с доступом к аккаунту и маркетинг в Бразилии'

In [34]:
from IPython.display import Markdown
Markdown(meeting_name_and_summary.meeting_summary)

Матт Льюис не смог войти в аккаунт, использовав чужой логин. Дмитрий Гранкин обсуждает свои физические активности, заметив увеличение веса на 6 кг, что повлияло на его бег. Дмитрий начал маркетинг в Бразилии, провел исследование конкурентов и заметил, что многие работают только в США. Он разработал движок для генерации статей на основе SEO, собирая ключевые слова и создавая статьи, что может увеличить трафик. Дмитрий столкнулся с трудностями при открытии счета в Interactive Brokers, требующими повторные подтверждения. Матт работает над улучшением своего блога для привлечения клиентов, создавая контент и видео. Обсуждается необходимость utility bill для открытия счета.

In [33]:
meeting_name_and_summary.meeting_summary

'Матт Льюис не смог войти в аккаунт, использовав чужой логин. Дмитрий Гранкин обсуждает свои физические активности, заметив увеличение веса на 6 кг, что повлияло на его бег. Дмитрий начал маркетинг в Бразилии, провел исследование конкурентов и заметил, что многие работают только в США. Он разработал движок для генерации статей на основе SEO, собирая ключевые слова и создавая статьи, что может увеличить трафик. Дмитрий столкнулся с трудностями при открытии счета в Interactive Brokers, требующими повторные подтверждения. Матт работает над улучшением своего блога для привлечения клиентов, создавая контент и видео. Обсуждается необходимость utility bill для открытия счета.'